### Load in the data for the new study after preprocessing (in the file: data_both_registers.ipynb)

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import warnings

from encoding import *
from rubins_rules import *
from Survival_functions import *

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)

# Replace the path with the actual path to your file
file_path = '../../../Both/new_study.xlsx'
df = pd.read_excel(file_path, index_col='PATNO')
df = df.rename(columns={'OS (days)': 'time'})
df = df.rename(columns={'Status': 'status'})

df['status'] = df['status'].map({'Dead': True, 'Alive': False})

print(df.shape)
df.columns

In [ ]:
df.index

In [ ]:
patients_with_false_status = df[df['status'] == False].index.tolist()
patients_with_false_status

In [ ]:
def find_categorical_features(df):
    categorical_features = [column for column in df.columns if df[column].dtype == 'object']
    return categorical_features

categorical_features = find_categorical_features(df)
features_df = pd.DataFrame(index=categorical_features)

# Add column 'Variables' containing unique values for each feature
variables_list = []
for feature in categorical_features:
    unique_values = df[feature].unique()
    variables_list.append("/ ".join(str(val) for val in unique_values))

features_df['Variables'] = variables_list
features_df.T

### Load in all packages

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RepeatedStratifiedKFold
mcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=173637)
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.metrics import (integrated_brier_score)
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer


In [ ]:
coxnet_df = df.copy()
X, y, tuple_y, target_columns = x_y_baseline(coxnet_df)

### TargetEncoding

Target Encoder transfer NaN values into values, so we must iterate through the features that must target encode, remove rows that have NaN and do a TargetEncoder.

- TargetEncoder used not the information about the others featues, it just use the target value to calculate on


TargetEncoding is done in the function Baseline_target_encoding(), that are defines in the file "encoding.py". The columns that must be encoded by using TargetEncoding is called "target_columns" here in this notebook. 

# COXNET


### Encoding of ordinal and binary catogorical variables

In the function x_y_baseline() all ordinal variables and binary catogorical variables are enocoded. And we also defines which columns that must be TargetEncoded (nominal columns)
- Binary variables without missing values: get_dummies encoded with `drop_first=True`. Got the same dimension. 
- Binary variables with missing values: encoded manually to 0 and 1 using `map`. 

The function x_y_baseline also split the dataframe to X and y. 

In [ ]:
coxnet_df = df.copy()
X, y, tuple_y, target_columns = x_y_baseline(coxnet_df)

### Model

Loop iteratively through all the hyperparameters. Fit on the given survival analysis model with the given hyperparamet (s). Inside the repeated cross-validation (MCV), we use the function Baseline_target_encoding(). This use TargetEncoding on the nominal columns for both training and testing set. Also in this function we also do this steps for both training and testing set:

- Scaling
- KNN Imputer with `n_neighbors=10`
- Scaling back to original (`inverse_transform`)
- Binary variables with missing values earlier are then rounded to the nearest whole number, resulting in either 0 or 1. Then these numbers are mapped back to their original categorical values before encoding them using get dummies with `drop_first=True`.
- Scale both training and testing set. 

It returns $X_{train}$ and $X_{test}$ that are finally preprocessed without any missing values and are now ready be used in the models. 

The code calculate:
- the integrated brier score (mean over all folds)
- concordance for test and train set (mean over all folds)
- Permutation importance 

for all hyperparameters or all comibinations of all hyperparameters. 

In [ ]:
# Repeated Statified cross validation for Coxnet model in sksurv
alphas = [0, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 3, 5, 10, 20, 50, 70, 100, 200, 500, 700, 1000]
#alphas = [0, 0.0001, 0.0003, 0.0005, 0.0007, 0.001, 0.003,  0.006, 0.007, 0.008, 0.009,
#          0.01, 0.03, 0.05, 0.07, 0.1, 0.3, 0.5, 0.7, 1, 3, 5, 10, 20, 50, 70, 100, 200, 500, 700, 1000]

l1_ratios = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#l1_ratios = [0.0001, 0.001, 0.01, 0.1]



results_coxnet = {}
feature_importance_coxnet = {}
coefficients_coxnet = {}
conc_coxnet = {}

for l1_ratio in l1_ratios:
    for alpha in alphas:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            
            coxnet = CoxnetSurvivalAnalysis(l1_ratio=l1_ratio, alphas=[alpha], fit_baseline_model=True)
            conc_train = []
            conc_test = []
            brier = []
            permut = []
            coef = []
            feature_importance = []
            
            print(f'l1_ratio: {l1_ratio}')
        
            for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
                X_train, X_test = X.iloc[train], X.iloc[test]
                y_train, y_test = y[train], y[test]
                
                X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
                # fix the times            
                times_train_min = y_train['time'].min()
                times_train_max = y_train['time'].max()
                times_train = np.arange(0, times_train_max)
                times_test_min = y_test['time'].min()
                times_test_max = y_test['time'].max()
                if times_test_max > times_train_max:
                    y_test_red_index = y_test['time'] <= times_train_max
                    y_test = y_test[y_test_red_index]
                    X_test = X_test[y_test_red_index]
                    times_test_max = y_test['time'].max()
                times_test = np.arange(times_test_min, times_test_max)

                
                coxnet.fit(X_train, y_train)
                
                # Compute the C-index for test data and train data
                conc_train.append(coxnet.score(X_train, y_train))
                conc_test.append(coxnet.score(X_test, y_test))

                # Integrated Brier Score
                surv_prob_test = np.row_stack([fn(times_test) for fn in coxnet.predict_survival_function(X_test)])
                brier.append(integrated_brier_score(y_train, y_test, surv_prob_test, times_test))

                importance = permutation_importance(coxnet,
                                                    X_test,
                                                    y_test,
                                                    n_repeats=10,
                                                    random_state=1)
                permut.append(importance.importances_mean)

                feature_importance.append(importance)
                coef.append(coxnet.coef_)
        
            feature_importance_coxnet[(alpha, l1_ratio)] = feature_importance
            coefficients_coxnet[(alpha, l1_ratio)] = coef

            # Evaluate and record the results after each alpha and l1_ratio combination
            avg_conc_test = np.mean(conc_test)
            avg_conc_train = np.mean(conc_train)
            avg_brier = np.mean(brier)
            avg_permut = np.mean(permut)

            results_coxnet[(alpha, l1_ratio)] = [avg_conc_test, avg_conc_train, avg_brier, avg_permut]

            conc_coxnet[(alpha, l1_ratio)] = conc_test

result = [{
    'Alpha': alpha,
    'L1 Ratio': l1_ratio,
    'Conc test': avg_conc_test,
    'Conc train': avg_conc_train,
    'Brier Score': avg_brier,
    'Permut': avg_permut
} for (alpha, l1_ratio), (avg_conc_test, avg_conc_train, avg_brier, avg_permut) in results_coxnet.items()]

# Create the DataFrame
results_coxnet = pd.DataFrame(result)


### Sort the result by the best Concordance (test)

In [ ]:
scores_coxnet = results_coxnet.sort_values(by='Conc test', ascending=False).reset_index(drop=True)

# Print out the sorted DataFrame
scores_coxnet.head(10)

### Model performance and feature selection across different alpha values with different L1 ratios. 

Use the function extract_alpha_coxnet(), that are in the Survival_functions.py. This function filters the Coxnet results for a given L1 ratio and computes the average coefficients for each unique alpha value within that L1 ratio. 
It then identifies the non-zero coefficients, constructs a dataframe mapping alphas to the number of selected features, and merges it with the input 
dataframe to obtain various model metrics.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Your l1_ratios and figure setup
l1_ratios = [0.0001, 0.001, 0.01, 0.1]
fig, axs = plt.subplots(2, 2, figsize=(13, 8))  # Setup for 4 subplots

# Iterate through l1_ratios and create subplots
for i, l1_ratio in enumerate(l1_ratios):
    alphas, _, conc_train, conc_test, brier, num_features, max_features, _ = extract_alpha_coxnet(
        results_coxnet, coefficients_coxnet, l1_ratio_value=l1_ratio, feature_names=X_train.columns)
    
    ax = axs[i // 2, i % 2]  # Determine the subplot
    x = np.arange(len(alphas))  # X locations for the groups
    
    # Plot concordance for train and test as lines
    ax.plot(x, brier, color='#FF6347', label='Brier Score')  
    ax.plot(x, conc_test, color='dodgerblue', label='C-Index (Test)')   
    ax.plot(x, conc_train, color='#00008B', label='C-Index (Train)')
    
    # Set x-axis labels and ticks
    ax.set_xticks(x)
    ax.set_xticklabels(alphas, rotation=45)
    ax.set_xlabel('Alpha')
    ax.set_ylabel('Score')
    ax.tick_params(axis='y')

    # Create a twin y-axis for the number of features
    ax2 = ax.twinx()
    ax2.plot(x, num_features, linestyle='--', color='g', label='Number of Features')
    ax2.set_ylabel('Number of features', color='g')
    ax2.tick_params(axis='y', labelcolor='g')
    #ax2.set_ylim(0, 50)

    ax.set_title(f'L1 ratio: {l1_ratio}', fontsize=15, style='italic')
    ax.grid()


# Creating a unified legend for the entire figure
labels = ['IBS', 'C-index Test', 'C-index Train', 'Number of Features']
colors = ['#FF6347', 'dodgerblue', '#00008B', 'g']
lines = [plt.Line2D([0], [0], color=color, marker='o', linestyle='-') for color in colors]
plt.figlegend(lines, labels, bbox_to_anchor=(0.73, 0.99), ncol=4, labelspacing=0.)

# Adjust layout and titles
plt.tight_layout(rect=[0, 0.05, 1, 0.95])  # Adjust the rect parameter as needed
#plt.suptitle('Coxnet Model Analysis Across Various L1 Ratios', fontsize=20, y=1.05)
plt.subplots_adjust(top=0.9, bottom=0.1)  # Adjust top and bottom to make space for title and legend
plt.show()


### Plot Alpha vs weights (concordance and brier score)

In [ ]:
l1_ratio = scores_coxnet['L1 Ratio'].iloc[0]

alphas, l1_ratio, conc_train, conc_test, brier, num_features, max_features, features_alpha = extract_alpha_coxnet(results_coxnet, 
                                                                                                                  coefficients_coxnet, 
                                                                                                                  l1_ratio_value=l1_ratio,
                                                                                                                  feature_names=X_train.columns)


In [ ]:
all_features = ["Absolute Neutrophil Count", "Chemotherapy Type"]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n_features = len(all_features)
n_cols = 2
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5.5, n_rows * 3.5), constrained_layout=True)
axes_flat = axes.flatten()

# Initialize the min and max coefficient values
min_coefficient = float('inf')
max_coefficient = float('-inf')

# First pass: find the global min and max across all plots
for feature_index, feature_name in enumerate(X_train.columns):
    if feature_name in all_features:
        feature_coefficients = []

        for alpha in alphas:
            specific_coefficients = coefficients_coxnet[(alpha, l1_ratio)]
            coefficients_array = np.array(specific_coefficients)
            mean_list = np.mean(coefficients_array, axis=0)

            if feature_index < len(mean_list):
                feature_coefficients.append(mean_list[feature_index])

        if feature_coefficients:
            # Update the global min and max
            min_coefficient = min(min_coefficient, min(feature_coefficients))
            max_coefficient = max(max_coefficient, max(feature_coefficients))

# Initialize a counter for the number of plotted features
plot_count = 0
# Second pass: plot using the global min and max
for feature_index, feature_name in enumerate(X_train.columns):
    if feature_name in all_features:
        feature_coefficients = []

        for alpha in alphas:
            specific_coefficients = coefficients_coxnet[(alpha, l1_ratio)]
            coefficients_array = np.array(specific_coefficients)
            mean_list = np.mean(coefficients_array, axis=0)

            if feature_index < len(mean_list):
                feature_coefficients.append(mean_list[feature_index])

        if feature_coefficients:
            ax = axes_flat[plot_count]
            ax.plot(alphas, feature_coefficients, marker='.', color='g')
            ax.set_xlabel('Alpha')
            ax.set_title(feature_name)
            ax.set_xscale('log')
            ax.set_ylabel('Coefficient', color='g')
            ax.tick_params(axis='y', labelcolor='g')
            ax.grid(True)

            # Set the same y-axis limits for coefficient plots
            ax.set_ylim(min_coefficient, max_coefficient)

            ax2 = ax.twinx()
            ax2.plot(alphas, conc_test, color='dodgerblue')
            ax2.plot(alphas, brier, color='salmon')
            ax2.set_ylabel('Score')
            ax2.tick_params(axis='y')
            ax2.set_ylim(0, 1)

            handles1, labels1 = ax.get_legend_handles_labels()
            handles2, labels2 = ax2.get_legend_handles_labels()
            handles = handles1 + handles2
            labels = labels1 + labels2
            ax.legend(handles, labels, loc='best')

            plot_count += 1  # Increment the plotted feature counter

# Hide unused axes
for i in range(plot_count, n_rows * n_cols):
    fig.delaxes(axes_flat[i])

# Creating a unified legend for the entire figure
labels = ['Coefficient', 'C-index Test', 'IBS']
colors = ['g', 'dodgerblue', 'salmon', ]
lines = [plt.Line2D([0], [0], color=color, marker='o', linestyle='-') for color in colors]
plt.figlegend(lines, labels, bbox_to_anchor=(0.69, 1.105), ncol=3, labelspacing=0.)

# Adjust layout and titles
#plt.suptitle('Coxnet Model Analysis Across Various L1 Ratios', fontsize=20, y=1.05)
#plt.subplots_adjust(top=0.9, bottom=0.1)  # Adjust top and bottom to make space for title and legend

plt.suptitle(f'(L1 ratio: {l1_ratio})', fontsize=12, style='italic', y=1.15)
plt.show()


In [ ]:
n_features = len(X_train.columns)
n_cols = 3  # Number of columns in the subplot grid
n_rows = (n_features + n_cols - 1) // n_cols  # Rounds up to ensure enough rows

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 6, n_rows * 4), constrained_layout=True)  # Adjust for layout

# Flatten the axes array for easy iteration
axes_flat = axes.flatten()

for feature_index, feature_name in enumerate(X_train.columns):
    feature_coefficients = []

    for alpha in alphas:
        specific_coefficients = coefficients_coxnet[(alpha, l1_ratio)]
        coefficients_array = np.array(specific_coefficients)
        mean_list = np.mean(coefficients_array, axis=0)

        if feature_index < len(mean_list):
            feature_coefficients.append(mean_list[feature_index])
        else:
            break
    
    if feature_coefficients:
        ax = axes_flat[feature_index]  # Use the flattened array of axes
        ax.plot(alphas, feature_coefficients, marker='.', color= 'dodgerblue', label='Coefficient')
        ax.set_xlabel('Alpha')
        ax.set_title(feature_name)
        ax.set_xscale('log')
        ax.set_ylabel('Coefficient', color = 'dodgerblue')
        ax.tick_params(axis='y', labelcolor='dodgerblue')

        ax.grid(True)

        ax2 = ax.twinx()
        ax2.plot(alphas, conc_test, color='g', label='C-Index')
        ax2.plot(alphas, brier, color='salmon', label='IBS')
        ax2.set_ylabel('Score')
        ax2.tick_params(axis='y')
        ax2.set_ylim(0, 1)

        # Handling the legend
        handles1, labels1 = ax.get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2
        labels = labels1 + labels2
        ax.legend(handles, labels, loc='best')  # You can adjust the location as needed

# Hide any unused axes if the number of features is not a multiple of n_cols
for i in range(feature_index + 1, n_rows * n_cols):
    fig.delaxes(axes_flat[i])

plt.suptitle(f'Regularization Effects: Balancing Model Permformance and Feature Selection (L1 ratio: {l1_ratio})', fontsize=20, style='italic')

plt.show()


### A horizontal bar chart of coefficients

In [ ]:
alpha = scores_coxnet['Alpha'].iloc[8]
l1_ratio = scores_coxnet['L1 Ratio'].iloc[8]
specific_coefficients = coefficients_coxnet[(alpha, l1_ratio)]
coefficients_array = np.array(specific_coefficients)
mean_list = np.mean(coefficients_array, axis=0)

best_coefs = pd.DataFrame(mean_list, index=X_train.columns, columns=["coefficient"])

non_zero = np.sum(best_coefs.iloc[:, 0] != 0)
print(f"Number of non-zero coefficients: {non_zero}")

non_zero_coefs = best_coefs.query("coefficient != 0")
coef_order = non_zero_coefs.abs().sort_values("coefficient").index

_, ax = plt.subplots(figsize=(10, 10))
non_zero_coefs.loc[coef_order].plot.barh(ax=ax, legend=False)
ax.set_xlabel("coefficient")
ax.set_title(f"Coxnet (alpha: {alpha:}, L1 Ratio: {l1_ratio})") 
ax.grid(True)
plt.show()

# COX PH

### Model

In [ ]:
X, y, tuple_y, target_columns = x_y_baseline(df)


In [ ]:
# Cross validation for CoxPH model in sksurv
alphas = [0, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 3, 5, 10, 20, 50, 70, 100, 200, 500, 700, 1000]

results_coxph = {}
feature_importance_ph = {}
coefficients_coxph = {}
conc_coxph = {}

for ind, alpha in enumerate(alphas):
        conc_train = []
        conc_test = []
        brier = []
        permut = []
        feat_impor = []
        coef = []

        coxph = CoxPHSurvivalAnalysis(alpha=alpha, ties="efron")

        print(f'alpha: {alpha}')
        
        for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
            X_train, X_test = X.iloc[train], X.iloc[test]
            y_train, y_test = y[train], y[test]
            
            X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
                
            # Train Model            
            times_train_min = y_train['time'].min()
            times_train_max = y_train['time'].max()
            times_train = np.arange(0, times_train_max)
            times_test_min = y_test['time'].min()
            times_test_max = y_test['time'].max()
            if times_test_max > times_train_max:
                y_test_red_index = y_test['time'] <= times_train_max
                y_test = y_test[y_test_red_index]
                X_test = X_test[y_test_red_index]
                times_test_max = y_test['time'].max()
            times_test = np.arange(times_test_min, times_test_max)

            
            coxph.fit(X_train, y_train)
            
            # Compute the C-index for test data and train data
            conc_train.append(coxph.score(X_train, y_train))
            conc_test.append(coxph.score(X_test, y_test))

            # Brier Score
            surv_prob_test = np.row_stack([fn(times_test) for fn in coxph.predict_survival_function(X_test)])
            brier.append(integrated_brier_score(y_train, y_test, surv_prob_test, times_test))

            importance = permutation_importance(coxph,
                                                X_test,
                                                y_test,
                                                n_repeats=10,
                                                random_state=1)
            permut.append(importance.importances_mean)

            feat_impor.append(importance)
            coef.append(coxph.coef_)
    
        feature_importance_ph[(alpha)] = feat_impor
        coefficients_coxph[(alpha)] = coef

        # Evaluate and record the results after each alpha
        avg_conc_test = np.mean(conc_test)
        avg_conc_train = np.mean(conc_train)
        avg_brier = np.mean(brier)
        avg_permut = np.mean(permut)

        results_coxph[(alpha)] = [avg_conc_test, avg_conc_train, avg_brier, avg_permut]
        conc_coxph[(alpha)] = conc_test

result = [{
    'Alpha': alpha,
    'Conc test': avg_conc_test,
    'Conc train': avg_conc_train,
    'Brier Score': avg_brier,
    'Permut': avg_permut
} for (alpha), (avg_conc_test, avg_conc_train, avg_brier, avg_permut) in results_coxph.items()]

# Create the DataFrame
results_coxph = pd.DataFrame(result)



### Sort the result by the best Concordance (test)

In [ ]:
scores_coxph = results_coxph.sort_values(by='Conc test', ascending=False).reset_index(drop=True)

# Print out the sorted DataFrame
scores_coxph.head(10)

### Plot Alpha vs weights (concordance and brier score)

In [ ]:
n_features = len(X_train.columns)
n_cols = 3  # Number of columns in the subplot grid
n_rows = (n_features + n_cols - 1) // n_cols  # Rounds up to ensure enough rows

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 6, n_rows * 4), constrained_layout=True)  # Adjust for layout

# Flatten the axes array for easy iteration
axes_flat = axes.flatten()

for feature_index, feature_name in enumerate(X_train.columns):
    feature_coefficients = []

    for alpha in alphas:
        specific_coefficients = coefficients_coxph[(alpha)]
        coefficients_array = np.array(specific_coefficients)
        mean_list = np.mean(coefficients_array, axis=0)

        if feature_index < len(mean_list):
            feature_coefficients.append(mean_list[feature_index])
        else:
            break
    
    if feature_coefficients:
        ax = axes_flat[feature_index]  # Use the flattened array of axes
        ax.plot(alphas, feature_coefficients, marker='.', color= 'dodgerblue', label='Coefficient')
        ax.set_xlabel('Alpha')
        ax.set_title(feature_name)
        ax.set_xscale('log')
        ax.set_ylabel('Coefficient', color = 'dodgerblue')
        ax.tick_params(axis='y', labelcolor='dodgerblue')

        ax.grid(True)

        ax2 = ax.twinx()
        ax2.plot(alphas, conc_test, color='g', label='Concordance')
        ax2.plot(alphas, brier, color='salmon', label='Brier score')
        ax2.set_ylabel('Score')
        ax2.tick_params(axis='y')
        ax2.set_ylim(0, 1)

        # Handling the legend
        handles1, labels1 = ax.get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles = handles1 + handles2
        labels = labels1 + labels2
        ax.legend(handles, labels, loc='best')  # You can adjust the location as needed

# Hide any unused axes if the number of features is not a multiple of n_cols
for i in range(feature_index + 1, n_rows * n_cols):
    fig.delaxes(axes_flat[i])

plt.suptitle(f'Regularization Effects: Balancing Model Accuracy and Feature Significance', fontsize=20, style='italic')

plt.show()


### Survival curves of all patients: Mean survival function across folds with variability and event observation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming best_alpha, best_l1_ratio, coxnet_best, and Baseline_target_encoding are defined
best_alpha = scores_coxph['Alpha'].iloc[1]

# Extract all PATNO numbers
patient_patnos = X.index

# Number of rows and columns for subplot
n_cols = 3
n_rows = len(patient_patnos) // n_cols + (len(patient_patnos) % n_cols > 0)

# Create a figure with subplots
plt.figure(figsize=(5 * n_cols, 4 * n_rows))

for j, patient_patno in enumerate(patient_patnos, start=1):
    survival_probabilities = []
    survival_times = []

    observed_event_time = df['time'].loc[patient_patno]
    
    for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
        X_train, X_test = X.iloc[train], X.iloc[test]
        y_train, y_test = y[train], y[test]
        
        X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
        
        coxph_best = CoxPHSurvivalAnalysis(alpha=alpha, ties="efron")
        coxph_best.fit(X_train, y_train)

        if patient_patno in X_test.index:
            patient_data = X_test.loc[[patient_patno]]
            surv_func = coxph_best.predict_survival_function(patient_data)
            survival_probabilities.append(surv_func[0].y)
            survival_times.append(surv_func[0].x)
    
    if survival_times:  # Check if there are any survival times collected
        ax = plt.subplot(n_rows, n_cols, j)
        
        # Plot each survival curve as a thin line
        for prob, times in zip(survival_probabilities, survival_times):
            plt.plot(times[:len(prob)], prob, color='dodgerblue', linewidth=0.5, alpha=0.5)
        
        # Compute mean and standard deviation
        min_length = min(len(arr) for arr in survival_times)
        truncated_probs = [arr[:min_length] for arr in survival_probabilities]
        stacked_probabilities = np.vstack(truncated_probs)
        mean_survival = np.mean(stacked_probabilities, axis=0)
        std_survival = np.std(stacked_probabilities, axis=0)
        common_times = np.array(survival_times[0][:min_length])
        
        # Plot the mean survival function as a thick line
        ax.plot(common_times, mean_survival, color='blue', linewidth=2, label="Mean Survival Function")
    
        # Add standard deviation shading
        ax.fill_between(common_times, mean_survival - std_survival, mean_survival + std_survival, color='grey', alpha=0.2, label="Std Dev")        
        ax.axvline(x=observed_event_time, color='red', linestyle='--', linewidth=2, label='Observed Event Time')
    
        ax.set_title(f"Patient {patient_patno}")
        ax.set_xlabel("Time in days")
        ax.set_ylabel("Survival probability")
        ax.legend()
        ax.grid(True)
    else:
        # If no data collected for a patient, this space will be empty
        print(f"No survival data collected for patient {patient_patno}.")

plt.tight_layout()
plt.show()


# RandomSurvivalForest

### RandomSurvivalForest

#### Model

In [ ]:
X, y, tuple_y, target_columns = x_y_baseline(df)


In [ ]:
n_estimators = [10, 20, 30, 40, 50]  
max_depths = [2, 3, 4, 5]
min_samples_splits = [2, 6, 8] 
min_samples_leafs = [1, 2, 3, 4, 5, 6]  

results_rf = {}
feature_importance_rf = {}

for n_estimator in n_estimators:
    for max_depth in max_depths:
        for min_samples_split in min_samples_splits:  
            for min_samples_leaf in min_samples_leafs: 
                rf = RandomSurvivalForest(
                    n_estimators=n_estimator, 
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    min_samples_leaf=min_samples_leaf,
                    random_state=173637)

                conc_train = []
                conc_test = []
                brier = []
                permut = []
                feat_impor = []
                coef = []
                
                #print(f'n_estimator: {n_estimator}, max_depth: {max_depth}, min_samples_split: {min_samples_split}, min_samples_leaf: {min_samples_leaf}')
                print(f'n_estimator: {n_estimator}, max_depth: {max_depth}')
                for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
                    X_train, X_test = X.iloc[train], X.iloc[test]
                    y_train, y_test = y[train], y[test]
                    
                    X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
                        
                    # fix the times            
                    times_train_min = y_train['time'].min()
                    times_train_max = y_train['time'].max()
                    times_train = np.arange(0, times_train_max)
                    times_test_min = y_test['time'].min()
                    times_test_max = y_test['time'].max()
                    if times_test_max > times_train_max:
                        y_test_red_index = y_test['time'] <= times_train_max
                        y_test = y_test[y_test_red_index]
                        X_test = X_test[y_test_red_index]
                        times_test_max = y_test['time'].max()
                    times_test = np.arange(times_test_min, times_test_max)

                    
                    rf.fit(X_train, y_train)
                    
                    # Compute the C-index for test data and train data
                    conc_train.append(rf.score(X_train, y_train))
                    conc_test.append(rf.score(X_test, y_test))

                    # Brier Score
                    surv_prob_test = np.row_stack([fn(times_test) for fn in rf.predict_survival_function(X_test)])
                    brier.append(integrated_brier_score(y_train, y_test, surv_prob_test, times_test))

                    importance = permutation_importance(rf,
                                                        X_test,
                                                        y_test,
                                                        n_repeats=10,
                                                        random_state=1)
                    permut.append(importance.importances_mean)

                    feat_impor.append(importance)

                feature_importance_rf[(n_estimator, max_depth, min_samples_split, min_samples_leaf)] = feat_impor

                # Evaluate and record the results after each n_estimator and max_depth combination
                avg_conc_test = np.mean(conc_test)
                avg_conc_train = np.mean(conc_train)
                avg_brier = np.mean(brier)
                avg_permut = np.mean(permut)

                results_rf[(n_estimator, max_depth, min_samples_split, min_samples_leaf)] = [avg_conc_test, avg_conc_train, avg_brier, avg_permut]

result = [{
    'n_estimator': n_estimator,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'Conc test': avg_conc_test,
    'Conc train': avg_conc_train,
    'Brier Score': avg_brier,
    'Permut': avg_permut
} for (n_estimator, max_depth, min_samples_split, min_samples_leaf), (avg_conc_test, avg_conc_train, avg_brier, avg_permut) in results_rf.items()]

# Create the DataFrame
results_rf = pd.DataFrame(result)

### Sort the result by the best Concordance (test)

In [ ]:
scores_rf = results_rf.sort_values(by='Conc test', ascending=False).reset_index(drop=True)

# Print out the sorted DataFrame
scores_rf.head(10)

### Survival curves of all patients: Mean survival function across folds with variability and event observation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming best_alpha, best_l1_ratio, coxnet_best, and Baseline_target_encoding are defined
n_estimator = scores_rf['n_estimator'].iloc[0]
max_depth = scores_rf['max_depth'].iloc[0]
min_samples_split = scores_rf['min_samples_split'].iloc[0]
min_samples_leaf = scores_rf['min_samples_leaf'].iloc[0]

# Extract all PATNO numbers
patient_patnos = X.index

# Number of rows and columns for subplot
n_cols = 3
n_rows = len(patient_patnos) // n_cols + (len(patient_patnos) % n_cols > 0)

# Create a figure with subplots
plt.figure(figsize=(5 * n_cols, 4 * n_rows))

for j, patient_patno in enumerate(patient_patnos, start=1):
    survival_probabilities = []
    survival_times = []

    observed_event_time = df['time'].loc[patient_patno]

    
    for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
        X_train, X_test = X.iloc[train], X.iloc[test]
        y_train, y_test = y[train], y[test]
        
        X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
        
        rf = RandomSurvivalForest(n_estimators=n_estimator, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
        rf.fit(X_train, y_train)

        if patient_patno in X_test.index:
            patient_data = X_test.loc[[patient_patno]]
            surv_func = rf.predict_survival_function(patient_data)
            survival_probabilities.append(surv_func[0].y)
            survival_times.append(surv_func[0].x)
    
    if survival_times:  # Check if there are any survival times collected
        ax = plt.subplot(n_rows, n_cols, j)
        
        # Plot each survival curve as a thin line
        for prob, times in zip(survival_probabilities, survival_times):
            plt.plot(times[:len(prob)], prob, color='dodgerblue', linewidth=0.5, alpha=0.5)
        
        # Compute mean and standard deviation
        min_length = min(len(arr) for arr in survival_times)
        truncated_probs = [arr[:min_length] for arr in survival_probabilities]
        stacked_probabilities = np.vstack(truncated_probs)
        mean_survival = np.mean(stacked_probabilities, axis=0)
        std_survival = np.std(stacked_probabilities, axis=0)
        common_times = np.array(survival_times[0][:min_length])
        
        # Plot the mean survival function as a thick line
        ax.plot(common_times, mean_survival, color='blue', linewidth=2, label="Mean Survival Function")
    
        # Add standard deviation shading
        ax.fill_between(common_times, mean_survival - std_survival, mean_survival + std_survival, color='grey', alpha=0.2, label="Std Dev")        
        ax.axvline(x=observed_event_time, color='red', linestyle='--', linewidth=2, label='Observed Event Time')
    
        ax.set_title(f"Patient {patient_patno}")
        ax.set_xlabel("Time in days")
        ax.set_ylabel("Survival probability")
        ax.legend()
        ax.grid(True)
    else:
        # If no data collected for a patient, this space will be empty
        print(f"No survival data collected for patient {patient_patno}.")

plt.tight_layout()
plt.show()


# Componentwise Gradient Boosting

### Model

In [ ]:
X, y, tuple_y, target_columns = x_y_baseline(df)


In [ ]:
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis
# loss: coxph
n_estimators = [10, 20, 30, 40, 50]
learning_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
subsamples = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

results_cgb = {}
feature_importance_cgb = {}
conc_cgb = {}

for learning_rate in learning_rates: 
        for n_estimator in n_estimators:
            for subsample in subsamples:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=UserWarning)
                    cgb = ComponentwiseGradientBoostingSurvivalAnalysis(n_estimators=n_estimator,
                                                                        learning_rate=learning_rate,
                                                                        subsample = subsample,
                                                                        random_state=173637)
                    conc_train = []
                    conc_test = []
                    brier = []
                    permut = []
                    feat_impor = []
                    coef = []
                    
                    print(f'learning_rate: {learning_rate}, n_estimator: {n_estimator}')
                
                    for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
                        X_train, X_test = X.iloc[train], X.iloc[test]
                        y_train, y_test = y[train], y[test]
                        
                        X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
                            
                        # fix the times            
                        times_train_min = y_train['time'].min()
                        times_train_max = y_train['time'].max()
                        times_train = np.arange(0, times_train_max)
                        times_test_min = y_test['time'].min()
                        times_test_max = y_test['time'].max()
                        if times_test_max > times_train_max:
                            y_test_red_index = y_test['time'] <= times_train_max
                            y_test = y_test[y_test_red_index]
                            X_test = X_test[y_test_red_index]
                            times_test_max = y_test['time'].max()
                        times_test = np.arange(times_test_min, times_test_max)

                        
                        cgb.fit(X_train, y_train)
                        
                        # Compute the C-index for test data and train data
                        conc_train.append(cgb.score(X_train, y_train))
                        conc_test.append(cgb.score(X_test, y_test))

                        # Brier Score
                        surv_prob_test = np.row_stack([fn(times_test) for fn in cgb.predict_survival_function(X_test)])
                        brier.append(integrated_brier_score(y_train, y_test, surv_prob_test, times_test))

                        importance = permutation_importance(cgb,
                                                            X_test,
                                                            y_test,
                                                            n_repeats=10,
                                                            random_state=1)
                        permut.append(importance.importances_mean)

                        feat_impor.append(importance)
                
                    feature_importance_cgb[(n_estimator, learning_rate, subsample)] = feat_impor

                    # Evaluate and record the results after each n_estimator and max_depth combination
                    avg_conc_test = np.mean(conc_test)
                    avg_conc_train = np.mean(conc_train)
                    avg_brier = np.mean(brier)
                    avg_permut = np.mean(permut)

                    results_cgb[(n_estimator, learning_rate, subsample)] = [avg_conc_test, avg_conc_train, avg_brier, avg_permut]
                    conc_cgb[(n_estimator, learning_rate, subsample)] = conc_test



result = [{
    'n_estimator': n_estimator,
    'learning_rate': learning_rate, 
    'subsample': subsample,
    'Conc test': avg_conc_test,
    'Conc train': avg_conc_train,   
    'Brier Score': avg_brier,
    'Permut': avg_permut
} for (n_estimator, learning_rate, subsample), (avg_conc_test, avg_conc_train, avg_brier, avg_permut) in results_cgb.items()]

# Create the DataFrame
results_cgb = pd.DataFrame(result)


### Sort the result by the best Concordance

In [ ]:
scores_cgb = results_cgb.sort_values(by='Conc test', ascending=False).reset_index(drop=True)

# Print out the sorted DataFrame
scores_cgb.head(10)

### Survival curves

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming best_alpha, best_l1_ratio, coxnet_best, and Baseline_target_encoding are defined
n_estimator_cgb = scores_cgb['n_estimator'].iloc[0]
subsample = scores_cgb['subsample'].iloc[0]
learning_rate = scores_cgb['learning_rate'].iloc[0]

# Extract all PATNO numbers
patient_patnos = X.index
#patient_patnos = [9007, 9024, 9040, 9065, 9056, 9033, 9008]

print(patient_patnos)

# Number of rows and columns for subplot
n_cols = 3
n_rows = len(patient_patnos) // n_cols + (len(patient_patnos) % n_cols > 0)

# Create a figure with subplots
plt.figure(figsize=(5 * n_cols, 4 * n_rows))

expected_survival_times = {}

for j, patient_patno in enumerate(patient_patnos, start=1):
    survival_probabilities = []
    survival_times = []

    observed_event_time = df['time'].loc[patient_patno]

    
    for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
        X_train, X_test = X.iloc[train], X.iloc[test]
        y_train, y_test = y[train], y[test]
        
        X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
        
        cgb = ComponentwiseGradientBoostingSurvivalAnalysis(n_estimators=n_estimator_cgb,
                                                            learning_rate=learning_rate,
                                                            subsample = subsample,
                                                            random_state=173637)
        cgb.fit(X_train, y_train)

        if patient_patno in X_test.index:
            patient_data = X_test.loc[[patient_patno]]
            surv_func = cgb.predict_survival_function(patient_data)
            survival_probabilities.append(surv_func[0].y)
            survival_times.append(surv_func[0].x)
    
    if survival_times:  # Check if there are any survival times collected
        ax = plt.subplot(n_rows, n_cols, j)
        
        # Plot each survival curve as a thin line
        for prob, times in zip(survival_probabilities, survival_times):
            plt.plot(times[:len(prob)], prob, color='dodgerblue', linewidth=0.5, alpha=0.5)
        
        # Compute mean and standard deviation
        min_length = min(len(arr) for arr in survival_times)
        truncated_probs = [arr[:min_length] for arr in survival_probabilities]
        stacked_probabilities = np.vstack(truncated_probs)
        mean_survival = np.mean(stacked_probabilities, axis=0)
        std_survival = np.std(stacked_probabilities, axis=0)
        common_times = np.array(survival_times[0][:min_length])

        # Now calculate the expected survival time for the patient
        expected_survival_time = approximate_integral(mean_survival, common_times)
        expected_survival_times[patient_patno] = expected_survival_time
        
        # Plot the mean survival function as a thick line
        ax.plot(common_times, mean_survival, color='blue', linewidth=2, label="Mean Survival Function")
    
        # Add standard deviation shading
        ax.fill_between(common_times, mean_survival - std_survival, mean_survival + std_survival, color='grey', alpha=0.2, label="Std Dev")
        ax.axvline(x=observed_event_time, color='red', linestyle='--', linewidth=2, label=f'Observed Event Time: {observed_event_time:.0f} days')
        ax.axvline(x=expected_survival_time, color='green', linestyle='--', linewidth=2, label=f'Expected Survival Time: {expected_survival_time:.0f} days')       
    
        ax.set_title(f"Patient {patient_patno}")
        ax.set_xlabel("Time in days")
        ax.set_ylabel("Survival probability")
        ax.legend()
        ax.grid(True)
    else:
        # If no data collected for a patient, this space will be empty
        print(f"No survival data collected for patient {patient_patno}.")

plt.tight_layout()
plt.show()


# Average Permutation importance for all models
Refer to the plot at the bottom of this section to view the average permutation importance across all models (Coxnet, CoxPH, RSF).

In [ ]:
# Coxnet
alpha_coxnet = scores_coxnet['Alpha'].iloc[8]
l1_ratio = scores_coxnet['L1 Ratio'].iloc[8]
best_feature_importance_coxnet = feature_importance_coxnet[(alpha_coxnet, l1_ratio)]
# CoxPH
alpha_coxph = scores_coxph['Alpha'].iloc[1]
best_feature_importance_ph = feature_importance_ph[(alpha_coxph)]
# RSF
n_estimator_rsf = scores_rf['n_estimator'].iloc[0]
max_depth_rsf = scores_rf['max_depth'].iloc[0]
min_samples_split_rsf = scores_rf['min_samples_split'].iloc[0]
min_samples_leaf_rsf = scores_rf['min_samples_leaf'].iloc[0]
best_feature_importance_rf = feature_importance_rf[(n_estimator_rsf, max_depth_rsf, min_samples_split_rsf, min_samples_leaf_rsf)]

n_estimator_cgb = scores_cgb['n_estimator'].iloc[0]
learning_rate_cgb = scores_cgb['learning_rate'].iloc[0]
subsample_cgb = scores_cgb['subsample'].iloc[0]
best_feature_importance_cgb = feature_importance_cgb[(n_estimator_cgb, learning_rate_cgb, subsample_cgb)]


### Coxnet

In [ ]:
import matplotlib.pyplot as plt

average_importance_coxnet = np.mean([imp.importances_mean for imp in best_feature_importance_coxnet], axis=0)

importances_coxnet = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance coxnet': average_importance_coxnet
})
top_importances_coxnet = importances_coxnet.sort_values(by='Importance coxnet', ascending=True).reset_index(drop=True)
#alpha_coxnet = scores_coxnet['Alpha'].iloc[8]
#l1_ratio = scores_coxnet['L1 Ratio'].iloc[8]


# Plot the bar chart
plt.figure(figsize=(10, 5))
plt.barh(top_importances_coxnet["Feature"].iloc[-20:], top_importances_coxnet["Importance coxnet"].iloc[-20:], color='skyblue')
plt.xlabel("Average Importance")
plt.ylabel('Feature')
plt.title(f"Average Permutation importance of Coxnet (alpha:{alpha_coxnet}, L1 ratio: {l1_ratio}) ")
plt.axvline(x=0, color="k", linestyle="--")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

average_importance_coxnet = np.mean([imp.importances_mean for imp in best_feature_importance_coxnet], axis=0)

importances_coxnet = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance coxnet': average_importance_coxnet
})
top_importances_coxnet = importances_coxnet.sort_values(by='Importance coxnet', ascending=True).reset_index(drop=True)


# Plot the bar chart
plt.figure(figsize=(10, 5))
plt.barh(top_importances_coxnet["Feature"].iloc[-30:], top_importances_coxnet["Importance coxnet"].iloc[-30:], color='skyblue')
plt.xlabel("Average Decrease in Concordance (Coxnet)")
plt.ylabel('Feature')
plt.title("Average Permutation importance ")
plt.axvline(x=0, color="k", linestyle="--")
plt.tight_layout()
plt.show()

### COX PH:

In [ ]:
import matplotlib.pyplot as plt

average_importance_ph = np.mean([imp.importances_mean for imp in best_feature_importance_ph], axis=0)

importances_ph = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance ph': average_importance_ph
})
top_importances_ph = importances_ph.sort_values(by='Importance ph', ascending=True).reset_index(drop=True)

# Plot the bar chart
plt.figure(figsize=(10, 5))
plt.barh(top_importances_ph["Feature"].iloc[-20:], top_importances_ph["Importance ph"].iloc[-20:], color='skyblue')
plt.xlabel("Average Decrease in Concordance ")
plt.ylabel('Feature')
plt.title("Average Permutation importance ")
plt.axvline(x=0, color="k", linestyle="--")
plt.tight_layout()
plt.show()

### RSF

In [ ]:
average_importance_rf = np.mean([imp.importances_mean for imp in best_feature_importance_rf], axis=0)

importances_rf = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance rsf': average_importance_rf
})
top_importances_rf = importances_rf.sort_values(by='Importance rsf', ascending=True).reset_index(drop=True)

# Plot the bar chart
plt.figure(figsize=(10, 5))
plt.barh(top_importances_rf["Feature"].iloc[-20:], top_importances_rf["Importance rsf"].iloc[-20:], color='skyblue')
plt.xlabel("Average Decrease in Concordance")
plt.ylabel('Feature')
plt.title("Average Permutation importance(RSF)")
plt.axvline(x=0, color="k", linestyle="--")
plt.tight_layout()
plt.show()

### CGB

In [ ]:
average_importance_cgb = np.mean([imp.importances_mean for imp in best_feature_importance_cgb], axis=0)

importances_cgb = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance cgb': average_importance_cgb
})
top_importances_cgb = importances_cgb.sort_values(by='Importance cgb', ascending=True).reset_index(drop=True)

# Plot the bar chart
plt.figure(figsize=(10, 5))
plt.barh(top_importances_cgb["Feature"].iloc[-20:], top_importances_cgb["Importance cgb"].iloc[-20:], color='skyblue')
plt.xlabel("Average Decrease in Concordance")
plt.ylabel('Feature')
plt.title("Average Permutation importance(CGB)")
plt.axvline(x=0, color="k", linestyle="--")
plt.tight_layout()
plt.show()

### Top 10 Feature Importance from Three Models

In [ ]:
import matplotlib.colors as mcolors

top_importances_coxnet = importances_coxnet.sort_values(by='Importance coxnet', ascending=False).reset_index(drop=True)
top_importances_ph = importances_ph.sort_values(by='Importance ph', ascending=False).reset_index(drop=True)
top_importances_rf = importances_rf.sort_values(by='Importance rsf', ascending=False).reset_index(drop=True)
top_importances_cgb = importances_cgb.sort_values(by='Importance cgb', ascending=False).reset_index(drop=True)


#merged_df = pd.merge(top_importances_coxnet, top_importances_ph, on='Feature', suffixes=('_coxnet', '_ph'))
merged_df = pd.merge(top_importances_coxnet, top_importances_ph, on='Feature')
merged_df = pd.merge(merged_df, top_importances_rf, on='Feature', how='outer')
merged_df = pd.merge(merged_df, top_importances_cgb, on='Feature', how='outer')

numeric_columns = ['Importance coxnet', 'Importance ph', 'Importance rsf', 'Importance cgb']
merged_df['Average_Importance'] = merged_df[numeric_columns].mean(axis=1)

top_merged_df = merged_df.nlargest(15, 'Average_Importance')

# Plotting
fig, ax = plt.subplots(figsize=(10, 5))

# We will have bars for each of the top 10 features, from each model
n = len(top_merged_df['Feature'])
index = np.arange(n)
bar_width = 0.15  # Reduce bar width to fit three bars

color_keys = list(mcolors.TABLEAU_COLORS.keys())  # Get color keys from TABLEAU_COLORS

bars1 = ax.bar(index - bar_width, top_merged_df['Importance coxnet'], bar_width, label='Coxnet', color=mcolors.TABLEAU_COLORS[color_keys[0]])
bars2 = ax.bar(index, top_merged_df['Importance ph'], bar_width, label='Cox PH', color=mcolors.TABLEAU_COLORS[color_keys[1]])
bars3 = ax.bar(index + bar_width, top_merged_df['Importance rsf'], bar_width, label='RSF', color=mcolors.TABLEAU_COLORS[color_keys[2]])
bars4 = ax.bar(index + 2*bar_width, top_merged_df['Importance cgb'], bar_width, label='CGB',color=mcolors.TABLEAU_COLORS[color_keys[3]])

ax.plot(index, top_merged_df['Average_Importance'], color=mcolors.TABLEAU_COLORS[color_keys[4]], marker= 'o', label='Average Importance')

# Adding labels, title, and legend
ax.set_xlabel('Feature')
ax.set_ylabel('Importance')
#ax.set_title('Top 10 Feature Importance from Three Models')
ax.set_xticks(index)
ax.set_xticklabels(top_merged_df['Feature'], rotation=45)
ax.legend()

# Showing the plot
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()  # Adjust layout to fit all labels
plt.show()


# Time dependent brier score between Coxnet/CoxPH/RSF/CGB

Plot the time dependent brier score for each model.
Since we use RepeatedKFold, this code takes the average for every 5 folds first, and then takes the average of these averages.

In [ ]:
from sksurv.metrics import brier_score

# Configurations for models
best_alpha_coxnet = scores_coxnet['Alpha'].iloc[8]
best_l1_ratio = scores_coxnet['L1 Ratio'].iloc[8]
best_alpha_coxph = scores_coxph['Alpha'].iloc[1]
n_estimator = scores_rf['n_estimator'].iloc[0]
max_depth = scores_rf['max_depth'].iloc[0]
min_samples_split = scores_rf['min_samples_split'].iloc[0]
min_samples_leaf = scores_rf['min_samples_leaf'].iloc[0]

n_estimator_cgb = scores_cgb['n_estimator'].iloc[0]
subsample = scores_cgb['subsample'].iloc[0]
learning_rate = scores_cgb['learning_rate'].iloc[0]


# Initialize RandomSurvivalForest
best_rsf = RandomSurvivalForest(
    n_estimators=n_estimator,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf
)

best_cgb = ComponentwiseGradientBoostingSurvivalAnalysis(n_estimators=n_estimator_cgb,
                                                         learning_rate=learning_rate,
                                                         subsample = subsample,
                                                         random_state=173637)
# Initialize storage for results
results = {
    "Coxnet": {"times": [], "scores": []},
    "CoxPH": {"times": [], "scores": []},
    "RSF": {"times": [], "scores": []},
    "CGB": {"times": [], "scores": []}
    }

# Run cross-validation for all models
for model_name, model in [
    ("Coxnet", CoxnetSurvivalAnalysis(l1_ratio=best_l1_ratio, alphas=[best_alpha_coxnet], fit_baseline_model=True)),
    ("CoxPH", CoxPHSurvivalAnalysis(alpha=best_alpha_coxph, ties="efron")),
    ("RSF", best_rsf), ("CGB", best_cgb)]:
    
    for train, test in mcv.split(X, tuple_y):
        X_train, X_test = X.iloc[train], X.iloc[test]
        y_train, y_test = y[train], y[test]

        X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)

        # We have to slice the times
        y_test = pd.DataFrame(y_test, columns=['status', 'time'])

        max_train = max(y_train["time"])
        times_x_axis_train = np.arange(0, max_train)
        y_test = y_test[y_test["time"] <= max_train]
        indices = y_test[y_test["time"] <= max_train].index
        X_test = X_test.iloc[indices]
        times_x_axis_test = np.arange(min(y_test["time"]), max(y_test["time"]))

        # Fit model
        model.fit(X_train, y_train)
        survival_pred = model.predict_survival_function(X_test)
        survival_preds = np.asarray([[fn(t) for t in times_x_axis_test] for fn in survival_pred])
        time, score = brier_score(y_train, y_test.to_records(index=False), survival_preds, times_x_axis_test)

        results[model_name]["times"].append(time)
        results[model_name]["scores"].append(score)

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))

color=mcolors.TABLEAU_COLORS[color_keys[0]]

colors = {"Coxnet": mcolors.TABLEAU_COLORS[color_keys[0]], "CoxPH": mcolors.TABLEAU_COLORS[color_keys[1]], "RSF": mcolors.TABLEAU_COLORS[color_keys[2]], "CGB": mcolors.TABLEAU_COLORS[color_keys[3]]}
color = {"Coxnet": "dodgerblue", "CoxPH": "lightsalmon", "RSF": 'limegreen', "CGB": "indianred"}
labels = {"Coxnet": "Coxnet mean Brier Score", "CoxPH": "CoxPH mean Brier Score", "RSF": "RSF mean Brier Score",
          "CGB": "CGB mean Brier Score"}

for model_name in ["Coxnet", "CoxPH", "RSF", "CGB"]:
    final_mean_briers = []
    final_common_times = []
    for i in range(0, len(results[model_name]["scores"]), 5):
        current_times = results[model_name]["times"][i:i + 5]
        current_scores = results[model_name]["scores"][i:i + 5]
        min_length = min(len(arr) for arr in current_times)
        truncated_scores = [arr[:min_length] for arr in current_scores]
        stacked_scores = np.vstack(truncated_scores)
        mean_scores = np.mean(stacked_scores, axis=0)
        final_mean_briers.append(mean_scores)
        final_common_times.append(np.array(current_times[0][:min_length]))
    
    # Plot each brier score as a thin line (dette kan fjernes!)
    for prob, times in zip(final_mean_briers, final_common_times):
        plt.plot(times[:len(prob)], prob, color=color[model_name], linewidth=0.5, alpha=0.5)

    min_length = min(len(arr) for arr in final_common_times)
    truncated_mean_briers = [arr[:min_length] for arr in final_mean_briers]
    stacked_mean_briers = np.vstack(truncated_mean_briers)
    overall_mean = np.mean(stacked_mean_briers, axis=0)
    overall_std = np.std(stacked_mean_briers, axis=0)
    common_times = np.array(final_common_times[0][:min_length])

    # Plot the mean and standard deviation for each model
    ax.plot(common_times, overall_mean, color=colors[model_name], linewidth=2, label=f"{labels[model_name]}")
    ax.fill_between(common_times, overall_mean - overall_std, overall_mean + overall_std, color=colors[model_name], alpha=0.2)

# Finalizing the plot with titles and labels
#ax.set_title("Comparison of Mean Time-dependent Brier Scores: Coxnet vs. CoxPH vs. RSF vs CGB", fontsize=16)
ax.set_xlabel("Time in days", fontsize=14)
ax.set_ylabel("Mean Time dependent Brier score", fontsize=14)
ax.legend(fontsize=12)
ax.grid(True)

# Retrieve model statistics
brier_coxnet = scores_coxnet['Brier Score'].iloc[8]
conc_coxnet = scores_coxnet['Conc test'].iloc[8]
conc_coxnet_train = scores_coxnet['Conc train'].iloc[8]
alpha_coxnet = scores_coxnet['Alpha'].iloc[8]
l1_coxnet = scores_coxnet['L1 Ratio'].iloc[8]
brier_coxph = scores_coxph['Brier Score'].iloc[1]
conc_coxph = scores_coxph['Conc test'].iloc[1]
conc_coxph_train = scores_coxph['Conc train'].iloc[1]
alpha_coxph = scores_coxph['Alpha'].iloc[1]
brier_rsf = scores_rf['Brier Score'].iloc[0]
conc_rsf = scores_rf['Conc test'].iloc[0]
conc_rsf_train = scores_rf['Conc train'].iloc[0]
n_estimator_rsf = scores_rf['n_estimator'].iloc[0]
max_depth_rsf = scores_rf['max_depth'].iloc[0]
min_samples_split_rsf = scores_rf['min_samples_split'].iloc[0]
min_samples_leaf_rsf = scores_rf['min_samples_leaf'].iloc[0]
brier_cgb = scores_cgb['Brier Score'].iloc[0]
conc_cgb = scores_cgb['Conc test'].iloc[0]
conc_cgb_train = scores_cgb['Conc train'].iloc[0]
n_estimator_cgb = scores_cgb['n_estimator'].iloc[0]
learning_rate_cgb = scores_cgb['learning_rate'].iloc[0]
subsample_cgb = scores_cgb['subsample'].iloc[0]

# Adding a text box with Brier scores and concordance indices


textstr = '\n'.join((
    f'Coxnet (alpha: {alpha_coxnet}, L1 ratio: {l1_coxnet}):',
    f'IBS: {brier_coxnet:.3f}',
    f'C-Index: (test: {conc_coxnet:.3f}, train:{conc_coxph_train:.3f})',
    f'--------------------------------------------------',
    f'CoxPH (alpha: {alpha_coxph}):',
    f'IBS: {brier_coxph:.3f}',
    f'C-Index: (test: {conc_coxph:.3f}, train: {conc_coxnet_train:.3f})',
    f'--------------------------------------------------',
    f'RSF: (n_estimator: {n_estimator_rsf}, max_depth: {max_depth_rsf}',
    f'min_samples_split: {min_samples_split_rsf}, min_samples_leaf: {min_samples_leaf_rsf})',
    f'IBS: {brier_rsf:.3f}',
    f'C-index: (test: {conc_rsf:.3f}, train: {conc_rsf_train:.3f})',
    f'-------------------------------------------------',
    f'CGB: (n_estimator: {n_estimator_cgb} ,learning_rate: {learning_rate_cgb}, subsample: {subsample_cgb})',
    f'IBS: {brier_cgb:.3f}',
    f'C-index: (test: {conc_cgb:.3f}, train: {conc_cgb_train:.3f})'
))

# Place a text box in upper left in axes coords
props = dict(boxstyle='round', facecolor='whitesmoke', alpha=0.5)
ax.text(0.72, 0.81, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)

plt.show()

# Surivival Curves for some pasients

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis

def approximate_integral(survival_probabilities, time_points):
    # Calculate time steps for each interval
    time_steps = np.diff(time_points)
    survival_probabilities_mid = (survival_probabilities[:-1] + survival_probabilities[1:]) / 2
    # Perform the integration using the trapezoidal rule
    return np.sum(survival_probabilities_mid * time_steps)


# Model initialization with provided parameters
coxnet_best = CoxnetSurvivalAnalysis(l1_ratio=scores_coxnet['L1 Ratio'].iloc[8], alphas=[scores_coxnet['Alpha'].iloc[8]], fit_baseline_model=True)
cgb_best = ComponentwiseGradientBoostingSurvivalAnalysis(n_estimators=scores_cgb['n_estimator'].iloc[0], learning_rate=scores_cgb['learning_rate'].iloc[0], subsample=scores_cgb['subsample'].iloc[0], random_state=173637)

#censored_patnos = [9001, 9033, 9056, 9065, 9099, 9110, 9111, 9114]
#patient_patnos_filtered = [patno for patno in patient_patnos if patno not in censored_patnos]

#patient_patnos_filtered = [9117, 9118, 9119, 9120, 9121]
patient_patnos_filtered = [9007, 9024, 9065, 9040]


# Plot configuration
n_cols = 2
n_rows = len(patient_patnos_filtered)
plt.figure(figsize=(5 * n_cols, 4 * n_rows))

# Process each patient for each model
for patient_index, patient_patno in enumerate(patient_patnos_filtered):

    # Initialize survival probabilities and times for both models
    survival_probabilities_coxnet = []
    survival_times_coxnet = []
    survival_probabilities_cgb = []
    survival_times_cgb = []

    observed_event_time = df['time'].loc[patient_patno]

    # Cross-validation and prediction for Coxnet model
    for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
        X_train, X_test = X.iloc[train], X.iloc[test]
        y_train, y_test = y[train], y[test]

        X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
            
        coxnet_best.fit(X_train, y_train)
        if patient_patno in X_test.index:
            patient_data = X_test.loc[[patient_patno]]
            surv_func = coxnet_best.predict_survival_function(patient_data)
            survival_probabilities_coxnet.append(surv_func[0].y)
            survival_times_coxnet.append(surv_func[0].x)

    # Cross-validation and prediction for CGB model
    for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
        X_train, X_test = X.iloc[train], X.iloc[test]
        y_train, y_test = y[train], y[test]

        X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
            
        cgb_best.fit(X_train, y_train)
        if patient_patno in X_test.index:
            patient_data = X_test.loc[[patient_patno]]
            surv_func = cgb_best.predict_survival_function(patient_data)
            survival_probabilities_cgb.append(surv_func[0].y)
            survival_times_cgb.append(surv_func[0].x)

    # Plotting for Coxnet model
    ax = plt.subplot(n_rows, n_cols, patient_index * n_cols + 1)
    if survival_times_coxnet:
        # Plot each survival curve as a thin line
        for prob, times in zip(survival_probabilities_coxnet, survival_times_coxnet):
            plt.plot(times[:len(prob)], prob, color='dodgerblue', linewidth=0.5, alpha=0.5)

        # Calculate mean and standard deviation
        min_length = min(len(arr) for arr in survival_times_coxnet)
        truncated_probs = [arr[:min_length] for arr in survival_probabilities_coxnet]
        stacked_probabilities = np.vstack(truncated_probs)
        mean_survival = np.mean(stacked_probabilities, axis=0)
        std_survival = np.std(stacked_probabilities, axis=0)
        common_times = np.array(survival_times_coxnet[0][:min_length])

        expected_survival_time = approximate_integral(mean_survival, common_times)

        ax.plot(common_times, mean_survival, color='blue', linewidth=2, label="Mean Survival Function (Coxnet)")
        ax.fill_between(common_times, mean_survival - std_survival, mean_survival + std_survival, color='grey', alpha=0.2, label="Std Dev (Coxnet)")
        ax.axvline(x=observed_event_time, color='red', linestyle='--', linewidth=2, label=f'Observed: {observed_event_time:.0f} days')
        ax.axvline(x=expected_survival_time, color='green', linestyle='--', linewidth=2, label=f'Expected: {expected_survival_time:.0f} days')

        ax.set_title(f"Patient {patient_patno} - Coxnet (alpha: 3, L1 Ratio: 0.01)")
        ax.set_xlabel("Time in days")
        ax.set_ylabel("Survival probability")
        ax.legend()
        ax.grid(True)
    else:
        ax.text(0.5, 0.5, "No data (Coxnet)", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

    # Plotting for CGB model
    ax = plt.subplot(n_rows, n_cols, patient_index * n_cols + 2)
    if survival_times_cgb:
        # Plot each survival curve as a thin line
        for prob, times in zip(survival_probabilities_cgb, survival_times_cgb):
            plt.plot(times[:len(prob)], prob, color='orange', linewidth=0.5, alpha=0.5)

        # Calculate mean and standard deviation
        min_length = min(len(arr) for arr in survival_times_cgb)
        truncated_probs = [arr[:min_length] for arr in survival_probabilities_cgb]
        stacked_probabilities = np.vstack(truncated_probs)
        mean_survival = np.mean(stacked_probabilities, axis=0)
        std_survival = np.std(stacked_probabilities, axis=0)
        common_times = np.array(survival_times_cgb[0][:min_length])

        expected_survival_time = approximate_integral(mean_survival, common_times)

        ax.plot(common_times, mean_survival, color='orange', linewidth=2, label="Mean Survival Function (CGB)")
        ax.fill_between(common_times, mean_survival - std_survival, mean_survival + std_survival, color='grey', alpha=0.2, label="Std Dev (CGB)")
        ax.axvline(x=observed_event_time, color='red', linestyle='--', linewidth=2, label=f'Observed: {observed_event_time:.0f} days')
        ax.axvline(x=expected_survival_time, color='green', linestyle='--', linewidth=2, label=f'Expected: {expected_survival_time:.0f} days')

        ax.set_title(f"Patient {patient_patno} - CGB")
        ax.set_xlabel("Time in days")
        ax.set_ylabel("Survival probability")
        ax.legend()
        ax.grid(True)
    else:
        ax.text(0.5, 0.5, "No data (CGB)", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.ensemble import ComponentwiseGradientBoostingSurvivalAnalysis

def approximate_integral(survival_probabilities, time_points):
    # Calculate time steps for each interval
    time_steps = np.diff(time_points)
    survival_probabilities_mid = (survival_probabilities[:-1] + survival_probabilities[1:]) / 2
    # Perform the integration using the trapezoidal rule
    return np.sum(survival_probabilities_mid * time_steps)

# Define the patients to analyze
#patient_patnos = [9007, 9008, 9024, 9040, 9065, 9056, 9033]
patient_patnos = X.index

#patient_patnos = [9007, 9024, 9065, 9035, 9040, 9100, 9008]
#patient_patnos = [9007, 9024, 9065, 9040]


# Model initialization with provided parameters
coxnet_best = CoxnetSurvivalAnalysis(l1_ratio=scores_coxnet['L1 Ratio'].iloc[0], alphas=[scores_coxnet['Alpha'].iloc[0]], fit_baseline_model=True)
cgb_best = ComponentwiseGradientBoostingSurvivalAnalysis(n_estimators=scores_cgb['n_estimator'].iloc[0], learning_rate=scores_cgb['learning_rate'].iloc[0], subsample=scores_cgb['subsample'].iloc[0], random_state=173637)

# Plot configuration
censored_patnos = [9001, 9033, 9056, 9065, 9099, 9110, 9111, 9114]
patient_patnos_filtered = [patno for patno in patient_patnos if patno not in censored_patnos]
print(patient_patnos_filtered)

# Plot configuration
n_cols = 2
n_rows = len(patient_patnos_filtered)
plt.figure(figsize=(5 * n_cols, 4 * n_rows))

# Process each patient for each model
for patient_index, patient_patno in enumerate(patient_patnos_filtered):

# Process each patient for each model
#for patient_index, patient_patno in enumerate(patient_patnos):
    models = [coxnet_best, cgb_best]
    for model_index, model in enumerate(models):
        survival_probabilities = []
        survival_times = []

        observed_event_time = df['time'].loc[patient_patno]

        # Cross-validation and prediction for each model
        for i, (train, test) in enumerate(mcv.split(X, tuple_y)):
            X_train, X_test = X.iloc[train], X.iloc[test]
            y_train, y_test = y[train], y[test]

            X_train, X_test = Preprocessing(X_train=X_train, X_test=X_test, y_train=y_train, target_columns=target_columns)
            
            model.fit(X_train, y_train)
            if patient_patno in X_test.index:
                patient_data = X_test.loc[[patient_patno]]
                surv_func = model.predict_survival_function(patient_data)
                survival_probabilities.append(surv_func[0].y)
                survival_times.append(surv_func[0].x)

        # Plotting
        ax = plt.subplot(n_rows, n_cols, patient_index * n_cols + model_index + 1)
        if survival_times:
            # Plot each survival curve as a thin line
            for prob, times in zip(survival_probabilities, survival_times):
                plt.plot(times[:len(prob)], prob, color='dodgerblue', linewidth=0.5, alpha=0.5)

            # Calculate mean and standard deviation
            min_length = min(len(arr) for arr in survival_times)
            truncated_probs = [arr[:min_length] for arr in survival_probabilities]
            stacked_probabilities = np.vstack(truncated_probs)
            mean_survival = np.mean(stacked_probabilities, axis=0)
            std_survival = np.std(stacked_probabilities, axis=0)
            common_times = np.array(survival_times[0][:min_length])

            expected_survival_time = approximate_integral(mean_survival, common_times)
            
            ax.plot(common_times, mean_survival, color='blue', linewidth=2, label="Mean Survival Function")
            ax.fill_between(common_times, mean_survival - std_survival, mean_survival + std_survival, color='grey', alpha=0.2, label="Std Dev")
            ax.axvline(x=observed_event_time, color='red', linestyle='--', linewidth=2, label=f'Observed: {observed_event_time:.0f} days')
            ax.axvline(x=expected_survival_time, color='green', linestyle='--', linewidth=2, label=f'Expected: {expected_survival_time:.0f} days')
            
            model_name = 'Coxnet ' if model_index == 0 else 'CGB'
            ax.set_title(f"Patient {patient_patno} - {model_name}")
            ax.set_xlabel("Time in days")
            ax.set_ylabel("Survival probability")
            ax.legend()
            ax.grid(True)
        else:
            ax.text(0.5, 0.5, "No data", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

plt.tight_layout()
plt.show()
